In [101]:
# portfolio VaR with correlated securities
# https://medium.com/financeexplained/learn-to-calculate-your-portfolios-value-at-risk-e1e2c5c68456
#https://financetrain.com/value-at-risk-of-a-portfolio/
#https://www.investopedia.com/ask/answers/033115/how-can-you-calculate-value-risk-var-excel.asp
#portVariance https://blog.quantinsti.com/calculating-covariance-matrix-portfolio-variance/

import pandas as pd
import numpy as np
import math
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

In [11]:
importDF = pd.read_csv(open('/Users/Glen/Documents/Python/AITrading/127compiledDailyData127.csv'))

In [12]:
importDF.head(15)

,date,open,high,low,close,volume,ticker
0,2020-06-08,9.7000,9.8300,9.4900,9.53,6083313.0,SRTY
1,2020-06-05,9.9600,10.2700,9.6500,10.13,11205729.0,SRTY
2,2020-06-04,11.6399,11.7800,11.2150,11.46,8088398.0,SRTY
3,2020-06-03,11.7200,11.8500,11.0900,11.41,9058773.0,SRTY
4,2020-06-02,12.3800,12.7900,12.1700,12.30,6600797.0,SRTY
5,2020-06-01,12.8800,13.0600,12.2100,12.62,6673939.0,SRTY
6,2020-05-29,13.1585,13.6750,12.8701,13.00,9948593.0,SRTY
7,2020-05-28,11.5256,12.9500,11.5256,12.76,7051185.0,SRTY
8,2020-05-27,12.4000,13.5588,11.8200,11.91,7613301.0,SRTY
9,2020-05-26,12.7100,13.2300,12.6500,13.17,5580446.0,SRTY


In [91]:
def generatePriceDF (df, tickers,priceType='close'):
    compiledData=pd.DataFrame()
    for ticker in tickers:
        tData=df[df['ticker']==ticker].reset_index()
        compiledData[ticker]=tData[priceType]
    return compiledData

In [123]:
tickers=['SRTY','CAL','TEN','OSUR']
pDF=generatePriceDF(importDF,tickers)
pChDF=pDF.pct_change(-1).dropna()
pChDF

,SRTY,CAL,TEN,OSUR
0,-0.059230,0.065751,0.227761,0.020803
1,-0.116056,0.073762,0.126652,-0.043056
2,0.004382,0.111241,0.093976,-0.040853
3,-0.072358,0.082383,0.070968,-0.070190
4,-0.025357,0.054813,0.058743,-0.142351
...,...,...,...,...
94,-0.000575,-0.021127,0.004965,0.008568
95,0.002882,-0.015257,-0.042776,0.004920
96,0.024203,-0.007343,-0.025023,-0.010949
97,0.009535,-0.008193,0.010300,-0.015569


In [124]:
assetMean=pChDF.mean().values
assetMean

array([-0.0008861 , -0.00078683,  0.00678215,  0.00482374])

In [125]:
assetCov=pChDF.cov().values
assetCov

array([[ 0.01069781, -0.00585852, -0.00697126, -0.00182859],
       [-0.00585852,  0.01209356,  0.00577145,  0.00011072],
       [-0.00697126,  0.00577145,  0.00966317,  0.00018351],
       [-0.00182859,  0.00011072,  0.00018351,  0.00410547]])

In [126]:
assetWeights=np.array([.1, .2,.3,.4])
portStd=math.sqrt(np.dot(np.dot(assetWeights.transpose(),assetCov),assetWeights))
portMean=np.dot(assetWeights.transpose(),assetMean)
print(portStd, portMean)

0.04552700950951771 0.0037181656445059684


In [129]:
z=1.65
VaR=portMean-z*portStd
VaR

-0.07140140004619826